In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [4]:
sc = SparkContext()
spark = SparkSession(sc)

24/01/19 09:55:56 WARN Utils: Your hostname, geekfox-nb resolves to a loopback address: 127.0.1.1; using 10.0.0.108 instead (on interface wlp2s0)
24/01/19 09:55:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/19 09:55:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/19 09:55:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df = spark.read.json('../sample_data/taxonomy_full.json')

In [6]:
df.show()

+--------------------+
|             reports|
+--------------------+
|[{GCA_963259705.1...|
+--------------------+


In [94]:
from pyspark.sql.functions import from_json

taxonomy_nodes = df.rdd.flatMap(lambda x: x.taxonomy_nodes)
taxonomy_nodes.take(1)

[Row(query=['51300'], taxonomy=Row(children=None, common_name='noctule', counts=[Row(count=37, type='COUNT_TYPE_GENE'), Row(count=13, type='COUNT_TYPE_PROTEIN_CODING'), Row(count=2, type='COUNT_TYPE_rRNA'), Row(count=22, type='COUNT_TYPE_tRNA')], descendent_with_described_species_names_count=None, has_described_species_name=True, lineage=[1, 131567, 2759, 33154, 33208, 6072, 33213, 33511, 7711, 89593, 7742, 7776, 117570, 117571, 8287, 1338369, 32523, 32524, 40674, 32525, 9347, 1437010, 314145, 9397, 30560, 9431, 51299], organism_name='Nyctalus noctula', rank='SPECIES', tax_id=51300))]

In [101]:
taxonomy = taxonomy_nodes.map(lambda x: x.taxonomy)
taxonomy.take(1)

[Row(children=None, common_name='noctule', counts=[Row(count=37, type='COUNT_TYPE_GENE'), Row(count=13, type='COUNT_TYPE_PROTEIN_CODING'), Row(count=2, type='COUNT_TYPE_rRNA'), Row(count=22, type='COUNT_TYPE_tRNA')], descendent_with_described_species_names_count=None, has_described_species_name=True, lineage=[1, 131567, 2759, 33154, 33208, 6072, 33213, 33511, 7711, 89593, 7742, 7776, 117570, 117571, 8287, 1338369, 32523, 32524, 40674, 32525, 9347, 1437010, 314145, 9397, 30560, 9431, 51299], organism_name='Nyctalus noctula', rank='SPECIES', tax_id=51300)]

In [115]:
def flat_counts(row):
   v_id = row[0]
   if row[1] is None:
      return [Row(tax_id=v_id, count=0, type='')]
   else:
       v_count = [ Row(tax_id=v_id, count=x['count'], type=x['type']) for x in row[1]]
       return v_count

counts = taxonomy.map(lambda x: (x.tax_id, x.counts))
counts_table = counts.flatMap(flat_counts)
counts_table.toDF().show()

+-------+-----+--------------------+
| tax_id|count|                type|
+-------+-----+--------------------+
|  51300|   37|     COUNT_TYPE_GENE|
|  51300|   13|COUNT_TYPE_PROTEI...|
|  51300|    2|     COUNT_TYPE_rRNA|
|  51300|   22|     COUNT_TYPE_tRNA|
| 117602|    0|                    |
|  77221|    0|                    |
| 258940|    0|                    |
|  94117|   13|     COUNT_TYPE_GENE|
|  94117|   13|COUNT_TYPE_PROTEI...|
|2871749|    0|                    |
|2778648|    0|                    |
| 187019|    0|                    |
| 258943|    0|                    |
|1578730|    0|                    |
| 412090|    0|                    |
|2069573|    0|                    |
|2510964|    0|                    |
|2742407|    0|                    |
|2742402|    0|                    |
|  27668|   37|     COUNT_TYPE_GENE|
+-------+-----+--------------------+


In [97]:
from pyspark.sql import Row

taxonomy_table = taxonomy.map(lambda x: Row(tax_id=x.tax_id, common_name=x.common_name, organism_name=x.organism_name, rank=x.rank))
taxonomy_table.take(1)

[Row(tax_id=51300, common_name='noctule', organism_name='Nyctalus noctula', rank='SPECIES')]

In [98]:
taxonomy_table.toDF().show()

+-------+--------------------+--------------------+----------+
| tax_id|         common_name|       organism_name|      rank|
+-------+--------------------+--------------------+----------+
|  51300|             noctule|    Nyctalus noctula|   SPECIES|
| 117602|                NULL|Miniopterus schre...|SUBSPECIES|
|  77221|                NULL|Rousettus aegypti...|SUBSPECIES|
| 258940|Himalayan whisker...| Myotis siligorensis|   SPECIES|
|  94117|little red flying...| Pteropus scapulatus|   SPECIES|
|2871749|                NULL|Eptesicus fuscus ...|SUBSPECIES|
|2778648|                NULL|  Laephotis robertsi|   SPECIES|
| 187019|Ryukyu tube-nosed...|     Murina ryukyuna|   SPECIES|
| 258943|  Rendall's serotine|Pseudoromicia ren...|   SPECIES|
|1578730|                NULL|Neoromicia cf. me...|   SPECIES|
| 412090|  Hanaki's dwarf bat| Pipistrellus hanaki|   SPECIES|
|2069573|Yok Don helmeted bat|Cassistrellus yok...|   SPECIES|
|2510964|                NULL|     Myotis araxenus|   S